In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import combinations
from scipy.sparse import csr_matrix

In [2]:
K = 10_000
basedir = "./lshtc"

In [3]:
def load_dataset(filename="train-remapped.csv", nmax=1_000_000_000_000):
    with open(filename, "r") as f:
        lines = f.readlines()

    class_set = set()
    labels = []
    features = []
    for l, line in tqdm(enumerate(lines), total=len(lines)-1):
        if l > nmax: break
        if l == 0: continue
        line = line.strip().split(" ")
        label = []
        feature = {}
        for element in line:
            if ":" not in element:
                element = int(element.replace(",", ""))
                class_set.add(element)
                label.append(element)
            else:
                feature_id = int(element.split(":")[0])
                feature_value = int(element.split(":")[1])
                feature[feature_id] = feature_value
        labels.append(label)
        features.append(feature)
    return class_set, features, labels

def filter_dataset(classes, X, Y, f):
    Xnew, Ynew = [], []
    for _x, _y in zip(X, Y):
        if f(_x, _y):
            Xnew.append(_x)
            Ynew.append(_y)
    classes_new = set([val for sublist in Ynew for val in sublist])
    return classes_new, Xnew, Ynew


In [4]:
classes, X, Y = load_dataset(f"{basedir}/train-remapped.csv")

2365437it [01:28, 26758.95it/s]                             


# Get the graph... 

In [5]:
def get_graph(hierarchy_file="hierarchy.txt"):
    with open(hierarchy_file, "r") as f:
        lines = f.readlines()
    G = nx.Graph()
    for l, line in tqdm(enumerate(lines), total=len(lines)-1):
        a, b = line.split(' ')
        a = int(a.strip())
        b = int(b.strip())
        if a in classes or b in classes:
            if a not in G.nodes():
                G.add_node(a)
            if b not in G.nodes():
                G.add_node(b)
            G.add_edge(a, b)
    return G

# Get the largest connected component in the graph... 
G = get_graph(f"{basedir}/hierarchy.txt")
G_components = [G.subgraph(cc_G) for cc_G in nx.connected_components(G)]
G_ours = G_components[np.argmax([len(G_c.nodes()) for G_c in G_components])] 
len(G_ours.nodes())

863261it [00:04, 212727.50it/s]                            


347434

# Filter the dataset by the largest connected component... 

In [6]:
# Filter by the largest CC... 
cc = set(G_ours.nodes())

Xnew, Ynew = [], []
for _x, _y in tqdm(zip(X, Y), total=len(X)-1):
    if len(set(_y) & cc) > 0:
        Xnew.append(_x)
        _y_new = []
        for y_indiv in _y:
            if y_indiv in cc:
                _y_new.append(y_indiv)
        Ynew.append(_y_new)
classes_new = set([val for sublist in Ynew for val in sublist])
# classes_new, Xnew, Ynew

2365436it [00:09, 241157.43it/s]                             


# Compute frequency list, get the top-K most frequent classes... 

In [7]:
def count_frequency(l):
    freq = {}
    for item in tqdm(l):
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return freq

l = [val for sublist in Ynew for val in sublist]
freq = count_frequency(l)

100%|██████████| 6062514/6062514 [00:03<00:00, 1888473.72it/s]


# For each point, choose the class with the highest frequency... 

In [8]:
classes, X, Y = classes_new, Xnew, Ynew

In [9]:
# Filter by the largest CC... 
cc = set(G_ours.nodes())

Xnew, Ynew = [], []
for _x, _y in tqdm(zip(X, Y), total=len(X)-1):
    Xnew.append(_x)
    _y_new = []
    _y_freqs = [freq[_yi] for _yi in _y]
    _y_new = [_y[np.argmax(_y_freqs)]]
    Ynew.append(_y_new)
classes_new = set([val for sublist in Ynew for val in sublist])
# classes_new, Xnew, Ynew

2117083it [00:15, 136016.97it/s]                             


In [10]:
class_list = np.array(list(classes_new))
freq_list = np.array([freq[c] for c in class_list])
topk_inds = np.argsort(freq_list)[::-1][:K]
topk_classes = class_list[topk_inds]
topk_freqs = freq_list[topk_inds]

# Filter by the top-K most frequent classes... 

In [11]:
classes, X, Y = classes_new, Xnew, Ynew

In [12]:
topk_classes_set = set(topk_classes)

Xnew, Ynew = [], []
for _x, _y in tqdm(zip(X, Y), total=len(X)-1):
    if len(set(_y) & topk_classes_set) > 0:
        Xnew.append(_x)
        _y_new = []
        for y_indiv in _y:
            if y_indiv in topk_classes_set:
                _y_new.append(y_indiv)
        Ynew.append(_y_new)
classes_new = set([val for sublist in Ynew for val in sublist])
# classes_new, Xnew, Ynew

2117083it [00:05, 397058.27it/s]                             


------------------------------------------------------------------------------------------------------------------------

# Get all shortest paths between the top-K most frequent classes (not subgraph)

In [ ]:
classes_new_list = list(classes_new)

In [ ]:
len(classes_new_list)

10000

In [ ]:
approx_steiner_tree = nx.algorithms.approximation.steiner_tree(
    G_ours, classes_new, method="mehlhorn")

#nx.algorithms.approximation.metric_closure(approx_steiner_tree)
len(approx_steiner_tree.nodes())
nodes = [node for (node, val) in approx_steiner_tree.degree()]
degrees = [val for (node, val) in approx_steiner_tree.degree()]

# lengths = dict(nx.all_pairs_dijkstra_path_length(approx_steiner_tree))
### Too slow!

#### Use O(|V|) all pairs shortest path trick for trees

# Define the root arbitrarily (in this case, the max degree vertex), 
# and orient the tree about this root
root = nodes[np.argmax(degrees)]
oriented_approx_steiner_tree = nx.dfs_tree(approx_steiner_tree, root)

# Get the LCA nodes (requires directedness)
pairs = combinations(classes_new_list, 2)
all_pairs_LCA = dict(nx.algorithms.tree_all_pairs_lowest_common_ancestor(
    oriented_approx_steiner_tree, pairs=pairs))

# Get distance between each node and root
d_root = nx.single_source_shortest_path_length(approx_steiner_tree, root)


c = K
d = np.zeros((c, c))
for i in tqdm(range(c)):
    for j in range(i+1, c):
        a = classes_new_list[i]
        b = classes_new_list[j]
        try:
            lca_overlap = all_pairs_LCA[(a, b)]
        except:
            lca_overlap = all_pairs_LCA[(b, a)]
        dist = d_root[a] + d_root[b] - (2 * d_root[lca_overlap])
        d[i, j] = dist
        d[j, i] = d[i, j]

#plt.matshow(d)

100%|██████████| 10000/10000 [01:32<00:00, 107.64it/s]


In [ ]:
# TODO save to disk

# Relabel classes to match matrix index...

todo

: 

: 

------------------------------------------------------------------------------------------

In [15]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=True)
X_sparse = v.fit_transform(Xnew)
Y_sparse = Ynew[:, 0]

In [58]:
#### Gets the indices for the first half of the K_train most frequent classes

K_train = 100

# Get the indices the the 100 most frequent classes
class_list = np.array(list(classes_new))
freq_list = np.array([freq[c] for c in class_list])
topk_inds = np.argsort(freq_list)[::-1][:K_train]
topk_classes = class_list[topk_inds]
topk_freqs = freq_list[topk_inds]
topk_classes_set = set(topk_classes)

inds = []
for i in range(len(Y_sparse)):
    y = Y_sparse[i]
    if y in topk_classes_set:
        inds.append(i)
inds = np.array(inds)

# randomize their order
inds = inds[np.random.permutation(len(inds))]

# take the first half 
inds = inds[:len(inds)//2]

# this is the training set 
X_train = X_sparse[inds]
Y_train = Y_sparse[inds]

# and the opposite is the test set
inds_rest = list(set(np.arange(Y_sparse.shape[0])) - set(inds))
X_test = X_sparse[inds_rest]
Y_test = Y_sparse[inds_rest]

In [55]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, Y_train)


KNeighborsClassifier(n_neighbors=10)

baseline

In [59]:
neigh.predict(X_test)